# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [173]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [174]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [175]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [176]:
df = data.groupby(["CustomerID","ProductName"]).sum().Quantity
df = pd.DataFrame(df)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [177]:
tabla= pd.pivot_table(df, values='Quantity', index=['ProductName'], columns=['CustomerID'], fill_value=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [223]:
s = squareform(pdist(tabla.T, 'euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [179]:
distances = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'euclidean'))), 
                         index=tabla.columns, columns=tabla.columns)
similarities = distances[200].sort_values(ascending=False)[1:].head()
similarities

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [180]:
dfs = df.loc[similarities.index]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [220]:
step6 = dfs.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
step6 = pd.DataFrame(step6)


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [219]:
step7 = step6[~step6.index.isin(df.loc[200].index)]

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [196]:
dic = {}
lista = list(data.CustomerID)

for y in lista:
    similarities = distances[y].sort_values(ascending=False)[1:6]
    dfs = df.loc[similarities.index]
    step6 = dfs.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
    step7 = step6[~step6.index.isin(df.loc[y].index)].head()
    dic[y] = step7.to_dict()
    
for y in dic:
    dic[y] = [x for x in dic[y]]
 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [217]:
reco = pd.DataFrame(dic).T
reco = reco.rename(index=str, columns={0:"Producto 1",1:"Producto 2",2:"Producto 3",3:"Producto 4",4:"Producto 5"})
reco.head(10)


,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural"
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,"Oregano - Dry, Rubbed",Halibut - Steaks
23548,Squid - Tubes / Tenticles 10/20,Flavouring - Orange,Bread - Raisin Walnut Oval,Sprouts - Baby Pea Tendrils,Banana Turning
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Beef - Inside Round
83106,Cocoa Butter,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Cheese - Mix,Sauce - Hollandaise
11253,Juice - Lime,Pomello,Tomatoes Tear Drop,"Potatoes - Instant, Mashed",Soup - Campbells Tomato Ravioli
35107,Bandage - Flexible Neon,Bread - French Baquette,Wine - Hardys Bankside Shiraz,Onion Powder,"Lentils - Red, Dry"
15088,Soupfoamcont12oz 112con,Soup Knorr Chili With Beans,Veal - Sweetbread,Wine - Fume Blanc Fetzer,Beef - Texas Style Burger
26031,Cheese - Mozzarella,"Lemonade - Natural, 591 Ml",Bananas,Quiche Assorted,Soup Knorr Chili With Beans


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [218]:
distances = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'correlation'))), 
                         index=tabla.columns, columns=tabla.columns)

dic = {}
lista = list(data.CustomerID)

for y in lista:
    similarities = distances[y].sort_values(ascending=False)[1:6]
    dfs = df.loc[similarities.index]
    step6 = dfs.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
    step7 = step6[~step6.index.isin(df.loc[y].index)].head()
    dic[y] = step7.to_dict()
    
for y in dic:
    dic[y] = [x for x in dic[y]]

reco = pd.DataFrame(dic).T
reco = reco.rename(index=str, columns={0:"Producto 1",1:"Producto 2",2:"Producto 3",3:"Producto 4",4:"Producto 5"})
reco.head(10)

,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
77352,Fondant - Icing,Grenadine,Mangoes,Flour - Whole Wheat,Lettuce - Frisee
40094,Truffle Cups - Brown,"Bread - Roll, Soft White Round",Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
23548,Olives - Stuffed,Wine - Charddonnay Errazuriz,Soup - Campbells Tomato Ravioli,Assorted Desserts,Juice - Orange
78981,"Tart Shells - Sweet, 4",Artichokes - Jerusalem,Sprouts - Baby Pea Tendrils,Chocolate - Dark,"Appetizer - Mini Egg Roll, Shrimp"
83106,Pastry - Cheese Baked Scones,Bouq All Italian - Primerba,Onions - Vidalia,Bread - Italian Roll With Herbs,"Turnip - White, Organic"
11253,Sauce - Demi Glace,"Nut - Chestnuts, Whole","Beef - Tenderlion, Center Cut","Mushroom - Trumpet, Dry","Soup - Canadian Pea, Dry Mix"
35107,Fond - Neutral,Snapple Lemon Tea,Truffle Cups - Brown,"Veal - Brisket, Provimi,bnls",Cheese Cloth No 100
15088,Browning Caramel Glace,Vaccum Bag 10x13,Pastry - Choclate Baked,Wine - Fume Blanc Fetzer,Skirt - 29 Foot
26031,Spinach - Baby,Foam Cup 6 Oz,General Purpose Trigger,Banana Turning,Smirnoff Green Apple Twist
